함수 메모장
파일 불러오기 : pd.read_csv('./tmdb_5000_credits.csv')
상위 n개 행 불러오기 : df2.head(n)
평균 : df2['행이름'].mean()
상위 100n% 값들만 필터링하기 : df2['행이름'].quantile(1 - n)
특정 행 검색 : df2.loc[조건식] (조건식 예시 : df2['행이름'] >= m)
특정 열 검색 : df2.loc['행이름']
각 행에 함수 적용 : q_movies.apply(함수명) (함수명 : 첫번째 parameter가 q_movies이여야함?)
행 정렬 : q_movies.sort_values(열name, ascending=False)



In [152]:
input1 = 'Family' 
input2 = 'Science_Fiction' 
input3 = 'Horror'
mix1 = input1 + ' ' + input2
mix2 = input1 + ' ' + input3
mix3 = input2 + ' ' + input3
mix4 = input1 + ' ' + input2 + ' ' + input3
mix5 = input1
mix6 = input2
mix7 = input3
mix8 = input1 + ' ' + input1 + ' ' + input2
from ast import literal_eval
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
movies = pd.read_csv('tmdb_5000_movies.csv')

movies_df = movies[['id','title', 'genres','keywords', 'vote_average', 'vote_count',
                 'popularity', 'overview', 'production_companies']]

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)
movies_df['production_companies'] = movies_df['production_companies'].apply(literal_eval)
# genres, keywords에서 id를 제외하고 name만 불러옴.
movies_df['genres'] = movies_df['genres'].apply(lambda x: [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [y['name'] for y in x])
movies_df['production_companies'] = movies_df['production_companies'].apply(lambda x: [y['name'] for y in x])

In [153]:
def connect(input):
    for i in range(len(input)):
        input[i] = input[i].replace(' ','_')
    return input

def cut_2_more(input, n = 2):
    #print(input)
    if(len(input) > n):
        input = input[:n]
    return input

def sum_pro_genre(input):
    return ' '.join(input['production_companies']) + ' ' + ' '.join(input['genres'])
movies_df['genres'] = movies_df['genres'].apply(connect)
movies_df['keywords'] = movies_df['keywords'].apply(connect)
movies_df['production_companies'] = movies_df['production_companies'].apply(connect)
#movies_df['genres'] = movies_df['genres'].apply(cut_2_more)
#movies_df['keywords'] = movies_df['keywords'].apply(cut_2_more)
movies_df['production_companies'] = movies_df['production_companies'].apply(cut_2_more)

print(movies_df['production_companies'][0])
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df['keywords_literal'] = movies_df['keywords'].apply(lambda x: (' ').join(x))
movies_df['production_companies_literal'] = movies_df.apply(sum_pro_genre, axis = 1)

['Ingenious_Film_Partners', 'Twentieth_Century_Fox_Film_Corporation']


In [154]:
movies_df['production_companies_literal']
#print(movies_df['production_companies_literal'])
print(type(movies_df['production_companies_literal']))

<class 'pandas.core.series.Series'>


In [155]:
# 모든 변수가 텍스트로 이루어졌으니 Countvectorizer로 feature 벡터화시키기
# 경우에 따라 추가적으로 Lemmatization 추가해 토큰화 전용 함수 추가도 가능!

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

keywords_vect = TfidfVectorizer().fit_transform(movies_df['keywords_literal'])
genres_vect = TfidfVectorizer().fit_transform(movies_df['genres_literal'])
production_companies_vect = TfidfVectorizer().fit_transform(movies_df['production_companies_literal'])

genre_info = 'Action Adventure Fantasy Science_Fiction Crime Drama Thriller Animation Family Western Comedy Romance Horror Mystery History War Music Documentary Foreign TV_Movie'
input_pandas = pd.Series([mix1,mix2,mix3,mix4,mix5,mix6 ,mix7,mix8, genre_info])
input_vect = TfidfVectorizer().fit_transform(input_pandas)

# 장르에 따른 영화별 코사인 유사도 추출
input_sim = cosine_similarity(input_vect, genres_vect)
#genre_sim = cosine_similarity(genres_vect, genres_vect)
keywords_sim = cosine_similarity(keywords_vect,keywords_vect)
production_companies_sim = cosine_similarity(production_companies_vect,production_companies_vect)
#tfid_sim = cosine_similarity(tfid_vect,tfid_vect)
# 3개만 유사도행렬값 추출해보기
print(genre_sim[:3])
print(genre_sim[0][3133])

[[0.75734461 0.89617067 0.66128327 ... 0.         0.         0.        ]
 [0.3638146  0.2834963  0.32705952 ... 0.38871387 0.         0.23237766]]
0.35072287622145343


In [156]:

def sort_similar(input,output):
    for i in range(len(input)):
        output[i] = input[i].argsort()[::-1]
production_companies_sim_idx = production_companies_sim.argsort()
keywords_sim_idx = keywords_sim.argsort()
genre_sim_idx = genre_sim.argsort()
input_sim_idx = input_sim.argsort()
sort_similar(input_sim,input_sim_idx)
sort_similar(production_companies_sim,production_companies_sim_idx)
sort_similar(genre_sim,genre_sim_idx)
sort_similar(keywords_sim,keywords_sim_idx)
print(input_sim_idx)


[[4601 3904 1053 ... 3110 3109 2401]
 [2044 4625 1396 ... 3025 3024    0]
 [4466 2841 3736 ... 3019 3016 2401]
 ...
 [3015 3312 1823 ... 3055 3054    0]
 [4601 2427 2601 ... 3110 3109 2401]
 [ 665  370 2681 ... 4068 4400 4611]]


In [157]:

#print(input_sim_idx[0])


print(movies_df.loc[input_sim_idx[0][0]]['title'])
print(movies_df.loc[input_sim_idx[1][0]]['title'])
print(movies_df.loc[input_sim_idx[2][0]]['title'])
print(movies_df.loc[input_sim_idx[3][0]]['title'])
print(movies_df.loc[input_sim_idx[4][0]]['title'])
print(movies_df.loc[input_sim_idx[5][0]]['title'])
print(movies_df.loc[input_sim_idx[6][0]]['title'])
print(movies_df.loc[input_sim_idx[7][0]]['title'])


"""
print(movies_df.loc[input_sim_idx[0][0]][['title','genres']])
print(movies_df.loc[input_sim_idx[1][0]][['title','genres']])
print(movies_df.loc[input_sim_idx[2][0]][['title','genres']])
print(movies_df.loc[input_sim_idx[3][0]][['title','genres']])
print(movies_df.loc[input_sim_idx[4][0]][['title','genres']])
print(movies_df.loc[input_sim_idx[5][0]][['title','genres']])
print(movies_df.loc[input_sim_idx[6][0]][['title','genres']])
print(movies_df.loc[input_sim_idx[7][0]][['title','genres']])
"""

Rotor DR1
The Little Vampire
Vessel
Prophecy
Rodeo Girl
Def-Con 4
Sinister 2
Rotor DR1


"\nprint(movies_df.loc[input_sim_idx[0][0]][['title','genres']])\nprint(movies_df.loc[input_sim_idx[1][0]][['title','genres']])\nprint(movies_df.loc[input_sim_idx[2][0]][['title','genres']])\nprint(movies_df.loc[input_sim_idx[3][0]][['title','genres']])\nprint(movies_df.loc[input_sim_idx[4][0]][['title','genres']])\nprint(movies_df.loc[input_sim_idx[5][0]][['title','genres']])\nprint(movies_df.loc[input_sim_idx[6][0]][['title','genres']])\nprint(movies_df.loc[input_sim_idx[7][0]][['title','genres']])\n"